In [ ]:
!pip install openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings
import openpyxl

# load data

In [ ]:
warnings.filterwarnings('ignore')

df_acci_b = pd.read_excel('/content/b1.xlsx')
df_resrc_b = pd.read_excel('/content/acci_info.xlsx')
df_acci_b['소방인력'] = '0'
df_acci_b['센터별_사고수'] = '0'

for i in range(len(df_acci_b)):
  for j in range(len(df_resrc_b)):
    if str(df_acci_b['서센터명'].iloc[i]) == str(df_resrc_b['center_name'].iloc[j]):
       df_acci_b['소방인력'].iloc[i] = df_resrc_b['person_per_center'].iloc[j]
       df_acci_b['센터별_사고수'].iloc[i] = df_resrc_b['acci_per_center'].iloc[j]

df_acci_b['소방인력'] = df_acci_b['소방인력'].astype(int)
df_acci_b['센터별_사고수'] = df_acci_b['센터별_사고수'].astype(int)
df_acci_b.shape

# add col(optional)

In [ ]:
acci_per_center_vle = []
ctr_nm_lst = []
acci_num_per_ctr_lst = []

acci_per_center_vle.append(df_acci_b['서센터명'].value_counts())
acci_per_center_vle_df = pd.DataFrame(acci_per_center_vle).transpose()

for i in range (len(acci_per_center_vle_df)):
  ctr_nm = re.sub(',', '', str(acci_per_center_vle_df.iloc[i]).split()[3])
  aci_num = re.sub(',', '', str(acci_per_center_vle_df.iloc[i]).split()[1])
  ctr_nm_lst.append(ctr_nm)
  acci_num_per_ctr_lst.append(aci_num)

ctr_nm_df = pd.DataFrame(ctr_nm_lst)
acci_num_per_ctr_df = pd.DataFrame(acci_num_per_ctr_lst)
acci_num_per_ctr_df = acci_num_per_ctr_df.astype(int)

center_df = pd.concat([ctr_nm_df, acci_num_per_ctr_df], ignore_index = True, axis=1)
center_df.columns = [['center_name', 'acci_per_center']]

In [ ]:
df_acci_b['1인당 담당 사고 수'] = 0.0
df_acci_b['Vi'] = 0.0

for i in range(len(df_acci_b)):
  for j in range(len(center_df)):
    if str(df_acci_b['서센터명'].iloc[i]) == str(center_df['center_name'].iloc[j]).split()[1]:
       aci_per_1 = round((center_df['acci_per_center'].iloc[j])/df_acci_b['소방인력'].iloc[i], 3)
       vi = round(1141/aci_per_1, 3)
       df_acci_b['1인당 담당 사고 수'].iloc[i] = aci_per_1
       df_acci_b['Vi'].iloc[i] = vi

df_acci_b['1인당 담당 사고 수'] = df_acci_b['1인당 담당 사고 수'].astype(float)
df_acci_b['Vi'] = df_acci_b['Vi'].astype(float)
df_acci_b.shape


In [ ]:
warnings.filterwarnings('ignore')

df_acci_b = pd.read_excel('/content/b21_kor_V2.xlsx')
df_resrc_b = pd.read_excel('/content/소방력_b23.xlsx')

df_acci_b['소방인력'] = '0'
acci_per_center_vle = []
ctr_nm_lst = []
acci_num_per_ctr_lst = []

acci_per_center_vle.append(df_acci_b['서센터명'].value_counts())
acci_per_center_vle_df = pd.DataFrame(acci_per_center_vle).transpose()

for i in range (len(acci_per_center_vle_df)):
  ctr_nm = re.sub(',', '', str(acci_per_center_vle_df.iloc[i]).split()[3])
  aci_num = re.sub(',', '', str(acci_per_center_vle_df.iloc[i]).split()[1])
  ctr_nm_lst.append(ctr_nm)
  acci_num_per_ctr_lst.append(aci_num)

ctr_nm_df = pd.DataFrame(ctr_nm_lst)
acci_num_per_ctr_df = pd.DataFrame(acci_num_per_ctr_lst)
acci_num_per_ctr_df = acci_num_per_ctr_df.astype(int)

center_df = pd.concat([ctr_nm_df, acci_num_per_ctr_df], ignore_index = True, axis=1)
center_df.columns = [['center_name','acci_per_center']]
center_df.to_excel('/content/acci_info.xlsx')

In [ ]:
center_df = pd.read_excel('/content/acci_info.xlsx')
center_df['person_per_center'] = '0'

for i in range(len(center_df)):
  for j in range(len(df_resrc_b)):
    if str(center_df['center_name'].iloc[i]) == str(df_resrc_b['서센터명'].iloc[j]):
      center_df['person_per_center'].iloc[i] = df_resrc_b['소방공무원'].iloc[j]

center_df.to_excel('/content/acci_info.xlsx')

In [ ]:
center_df.to_excel('/content/acci_info.xlsx')

# del outliers

In [ ]:
def compute_statistics(data, weight):
    if data.dtype == 'object':
        return {}  # 문자열 데이터인 경우 빈 딕셔너리 반환

    q_25 = data.quantile(q=0.25)
    q_75 = data.quantile(q=0.75)
    iqr = q_75 - q_25
    bound_l = q_25 - iqr * weight
    bound_u = q_75 + iqr * weight

    if data.min() < bound_l:
        bound_l = data.min()  # 최소값보다 작은 값은 최소값으로 설정

    return {
        "Lower Bound": bound_l,
        "Upper Bound": bound_u
    }

In [ ]:
del_col_lst = []

for col in df_acci_b.columns:
    if df_acci_b[col].dtypes != 'object':
        df_max = df_acci_b[col].describe()[7]
        df_min = df_acci_b[col].describe()[3]
        bounds = compute_statistics(df_acci_b[col], 1.5)

        if df_max > bounds.get("Upper Bound", df_max):
            del_col_lst.append(col)

        if df_min < bounds.get("Lower Bound", df_min):
            if col not in del_col_lst:
                del_col_lst.append(col)

In [ ]:
for i in del_col_lst:
    input_data = df_acci_b[i]
    bounds = compute_statistics(input_data, 1.5)
    outliers = input_data[(input_data < bounds.get("Lower Bound", input_data.min())) | (input_data > bounds.get("Upper Bound", input_data.max()))]  # 수정된 부분
    df_filt_outli = df_acci_b[~df_acci_b[i].isin(outliers)]
df_filt_outli.shape

# dummi & label encoder

In [ ]:
label_encoder = LabelEncoder()
df_enc = df_filt_outli.copy()

to_dummi_lst = [
    '화재유형명',
    '건물구조식',
    '건물구조조',
    '건물구조즙',
    '발화열원',
    '최초착화물',
    '발화기기',
    '연소확대물',
    '시설장소',
    ]

for col in to_dummi_lst:
        df_dummi = pd.get_dummies(df_enc[col],prefix=col)
        df_enc  = pd.concat([df_enc, df_dummi], axis=1)
        df_enc.drop(columns=[col], inplace=True)

for col in to_enc_lst:
      df_enc[f'{col}_encoded'] = label_encoder.fit_transform(df_enc[col])
      df_enc.drop(columns=[col], inplace=True)

print(df_enc.shape)

In [ ]:
df_enc.drop(labels=['서센터명'],axis=1,inplace=True)

# del nan

In [ ]:
columns_with_nan = df_enc.columns[df_enc.isnull().any()].tolist()
for col in columns_with_nan:
  df_enc.replace([np.inf, -np.inf], np.nan, inplace=True)
  df_enc[col] = df_enc[col].replace(np.nan, 0)

# scaler

In [ ]:
scaler = MinMaxScaler()

for col in df_enc.columns:
    df_minmax = scaler.fit_transform(df_enc[[col]])
    df_enc = pd.concat([df_enc, pd.DataFrame(df_minmax,  columns=[f'{col}_scld'])], axis=1)
    df_enc.drop(columns=[col], inplace=True)

df_scaled = df_enc

In [ ]:
df_scaled.to_excel('/content/b1_scld.xlsx')

# corr

In [ ]:
warnings.filterwarnings('ignore')
df = pd.read_excel('/content/b21_kor_v2.5.xlsx')
corr_df = df.corr()

In [ ]:
corr_df['재산피해금액'].sort_values(ascending=False)

In [ ]:
corr_df['화재진압시간'].sort_values(ascending=False)

In [ ]:
corr_df['소방인력'].sort_values(ascending=False)

# randint(optional)

In [ ]:
df = pd.read_excel('/content/b2.xlsx')
df.shape

In [ ]:
for col in df.columns:
  for i in range(38, 40):
      new_row = pd.DataFrame({f'{col}': [np.random.randint(df[col].min(), df[col].max() + 1, dtype=int)]})
      df = pd.concat([df, new_row], ignore_index=True)
df.shape